In [37]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from IPython.display import display, HTML
from collections import Counter
import pandas as pd
import numpy as np
import joblib
import h5py
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", None)

display(HTML("""
<style>
.dataframe td, .dataframe th {
    white-space: nowrap !important;
}
</style>
"""))


In [48]:
def get_reference_set_compounds(compounds):
    """
    Return a reference set of ChEMBL compound IDs from a compounds DataFrame.

    If the DataFrame has more than 10,000 rows, this returns a reduced reference
    set consisting of the first 5,000 and the last 5,000 `compound_chembl_id`
    values (as a list). Otherwise, it returns the full `compound_chembl_id`
    column.
    """
    if len(compounds) > 10000:
        return compounds['compound_chembl_id'][:5000].tolist() + compounds['compound_chembl_id'][-5000:].tolist()
    else:
        return compounds['compound_chembl_id']
    
def load_ecfp_subset_by_chembl_id(h5_path, chembl_id_set):
    """Load ECFP (Morgan count) fingerprints and return only requested ChEMBL IDs.

    Parameters
    ----------
    h5_path : str
        Path to the HDF5 file containing datasets "SMILES" and "X_morgan".
    chembl_id_set : set[str]
        ChEMBL IDs to keep.

    Returns
    -------
    dict[str, np.ndarray]
        Mapping {chembl_id: fingerprint (np.int8, shape (nBits,))}.
    """
    with h5py.File(h5_path, "r") as f:
        meta = f["SMILES"][:, 3].astype(str)
        fps  = f["X_morgan"][:]  # load all

    return {cid: fp for cid, fp in zip(meta, fps) if cid in chembl_id_set}

In [49]:
def get_pathogen_code(pathogen):
    return str(pathogen.split()[0][0] + pathogen.split()[1]).lower() if len(pathogen.split()) > 1 else pathogen.lower()

def condition_A(df):
    return df[ (df["dataset_type"].isin(["quantitative", "mixed"])) & 
               (df["cpds_qt"] >= 100) &
               (df["pos_qt"] >= 5) & 
               (df["ratio_qt"].between(0.01, 0.5, inclusive="both"))]

In [50]:
# Define root directory
root = "."

# List of pathogens to process
pathogens = ["Acinetobacter baumannii", "Candida albicans", "Campylobacter", "Escherichia coli", "Enterococcus faecium", "Enterobacter",
             "Helicobacter pylori", "Klebsiella pneumoniae", "Mycobacterium tuberculosis", "Neisseria gonorrhoeae", "Pseudomonas aeruginosa",
             "Plasmodium falciparum", "Staphylococcus aureus", "Schistosoma mansoni", "Streptococcus pneumoniae"]
pathogens = ["Acinetobacter baumannii", "Mycobacterium tuberculosis", "Klebsiella pneumoniae"]

# Create output directory
OUTPUT = os.path.join(root, "..", "output")

pathogen_to_datasets = {}

# For each pathogen
for pathogen in pathogens[1:2]:

    # Get pathogen code
    pathogen_code = get_pathogen_code(pathogen)
    pathogen_to_datasets[pathogen] = dict()

    # Get data information
    data_info = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_data.csv"))

    # Get compounds for pathogen
    compounds = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "compound_counts.csv.gz"))
    REFERENCE_SET = get_reference_set_compounds(compounds)
    compounds = set(compounds['compound_chembl_id'])
    print(f"Loading ECFP6s for pathogen: {pathogen}. {len(compounds)} compounds")

    # Loading Morgan fingerprints
    PATH_TO_ECFPs = os.path.join(root, "..", "output", "descriptors.h5")
    ecfps = load_ecfp_subset_by_chembl_id(PATH_TO_ECFPs, compounds)

Loading ECFP6s for pathogen: Mycobacterium tuberculosis. 132378 compounds


In [51]:
A = condition_A(data_info)

In [62]:
X_REF = np.array([ecfps[cid] for cid in REFERENCE_SET if cid in ecfps])

In [65]:
X_REF

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [53]:
AUROCS, STDS = [], []

# Iterate over assays A
for c, assay in A.iterrows():

    # Load varibles
    assay_id = assay.assay_id
    activity_type = assay.activity_type
    unit = assay.unit
    
    # Load data
    filename = "_".join([str(assay_id), str(activity_type), str(unit), "qt.csv.gz"])
    df = pd.read_csv(os.path.join(OUTPUT, pathogen_code, 'datasets', filename))

    # Prepare matrices
    X = np.array(df['compound_chembl_id'].map(ecfps).to_list())
    Y = np.array(df['bin'].tolist())

    print(f"Assay ID: {assay_id}, Activity type: {activity_type}, Unit: {unit}")
    print(X.shape, Y.shape)

    # Shuffle systematically
    rng = np.random.default_rng(42)   # fixed seed
    idx = rng.permutation(len(Y))
    X = X[idx]
    Y = Y[idx]

    RF = RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        n_jobs=8,
        random_state=42
    )

    # 4Fold CV
    skf = StratifiedKFold(n_splits=4)
    aurocs = []

    # For each fold
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, Y), 1):

        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]

        # fit model
        RF.fit(X_train, Y_train)

        # predicted probabilities for AUROC
        y_prob = RF.predict_proba(X_test)[:, 1]

        # compute AUROC
        auc = roc_auc_score(Y_test, y_prob)
        aurocs.append(auc)
        
        print(f"Fold {fold}: AUROC = {auc:.4f}")

    print(f"Mean AUROC: {np.mean(aurocs):.4f} ± {np.std(aurocs):.4f}")
    average_auroc = round(np.mean(aurocs), 3)
    stds = round(np.std(aurocs), 3)
    AUROCS.append(average_auroc)
    STDS.append(stds)





    # Train with full data
    # Save model

    break
    
    

Assay ID: CHEMBL4649948, Activity type: PERCENTEFFECT, Unit: %
(86589, 2048) (86589,)
Fold 1: AUROC = 0.6908
Fold 2: AUROC = 0.7246
Fold 3: AUROC = 0.7053
Fold 4: AUROC = 0.7144
Mean AUROC: 0.7088 ± 0.0124


In [61]:
joblib.dump(RF, os.path.join(OUTPUT, pathogen_code, 'models', 'A', filename.replace('.csv.gz', '_A.pkl')), compress=1)

['./../output/mtuberculosis/models/A/CHEMBL4649948_PERCENTEFFECT_%_qt_A.pkl']

In [69]:
Counter(RF.predict_proba(X_REF)[:,1].tolist())

Counter({0.0: 3304,
         0.01: 1699,
         0.02: 1240,
         0.03: 914,
         0.04: 711,
         0.05: 512,
         0.06: 358,
         0.07: 262,
         0.08: 159,
         0.09: 150,
         0.1: 97,
         0.11: 87,
         0.12: 56,
         0.13: 39,
         0.14: 34,
         0.63: 18,
         0.64: 18,
         0.61: 17,
         0.66: 16,
         0.16: 15,
         0.67: 14,
         0.58: 14,
         0.7: 14,
         0.17: 13,
         0.69: 13,
         0.57: 12,
         0.15: 11,
         0.18: 10,
         0.6: 10,
         0.21: 9,
         0.65: 9,
         0.59: 9,
         0.62: 8,
         0.68: 8,
         0.71: 8,
         0.23: 7,
         0.19: 7,
         0.72: 6,
         0.2: 5,
         0.54: 5,
         0.025: 5,
         0.015: 5,
         0.55: 4,
         0.74: 4,
         0.81: 3,
         0.79: 3,
         0.24: 3,
         0.25: 3,
         0.22: 3,
         0.0175: 2,
         0.066: 2,
         0.29: 2,
         0.49988095238